
# 数理最適化モジュール mypulp

> 混合整数問題をモデル化するためのモジュール pulp を商用パッケージGurobiと同じ文法で記述するためのラッパーモジュール

In [ ]:
#| default_exp mypulp

In [ ]:
#| include: false
%reload_ext autoreload
%autoreload 2

## インポート

In [ ]:
#| export
TEST = False
import math
import sys
import pulp

from pulp import (
    LpAffineExpression,
    LpConstraint,
    LpConstraintEQ,
    LpConstraintGE,
    LpConstraintLE,
    LpContinuous,
    LpInteger,
    LpProblem,
    LpVariable,
    lpSum,
    sys,
    lpDot,
    value,
    SCIP
)

#from pulp import PULP_CBC_CMD

quicksum = lpSum

## GRB（定数）クラス

In [ ]:
#| export
# {1: 'LOADED', 2: 'OPTIMAL', 3: 'INFEASIBLE', 4: 'INF_OR_UNBD', 5: 'UNBOUNDED', 6: 'CUTOFF', 7: 'ITERATION_LIMIT', 8: 'NODE_LIMIT', 9: 'TIME_LIMIT', 
# 10: 'SOLUTION_LIMIT', 11: 'INTERRUPTED', 12: 'NUMERIC', 13: 'SUBOPTIMAL', 14: 'INPROGRESS', 15: 'USER_OBJ_LIMIT'}
class GRB:
    class Status:
        OPTIMAL = 2
        INFEASIBLE = 3
        INF_OR_UNBD = 4
        UNBOUNDED = 5
        UNDEFINED = None

    CONTINUOUS = "C"
    INTEGER = "I"
    BINARY = "B"
    SEMICONT = "S"
    SEMIINT = "N"

    MINIMIZE = 1
    MAXIMIZE = -1
    INFINITY = 1e100
    MAXINT = 2147483647

    SOS_TYPE1 = 1
    SOS_TYPE2 = 2

## 区分的線形関数

In [ ]:
#| export
def gray(i):
    return i ^ i // 2

In [ ]:
#| export
def convex_comb_agg_log(model, var_list=[]):
    """add piecewise relation with a logarithmic number of binary variables

    using the convex combination formulation -- non-disaggregated.
    Parameters:
        - model: a model where to include the SOS type 2
        - vars : list of variables that declare the SOS type 2
    """
    K = len(var_list) - 1
    G = int(math.ceil((math.log(K) / math.log(2))))  # number of required bits
    g = {}
    for j in range(G):
        g[j] = model.addVar(vtype="B")
    model.addConstr(quicksum(var_list[k] for k in range(K + 1)) == 1)
    # binary variables setup
    for j in range(G):
        zeros, ones = [0], []
        # print j,"\tinit zeros:",zeros,"ones:",ones
        for k in range(1, K + 1):
            # print j,k,"\t>zeros:",zeros,"ones:",ones
            if (1 & gray(k) >> j) == 1 and (1 & gray(k - 1) >> j) == 1:
                ones.append(k)
            if (1 & gray(k) >> j) == 0 and (1 & gray(k - 1) >> j) == 0:
                zeros.append(k)
            # print j,k,"\tzeros>:",zeros,"ones:",ones

        # print j,"\tzeros:",zeros,"ones:",ones
        model.addConstr(quicksum(var_list[k] for k in ones) <= g[j])
        model.addConstr(quicksum(var_list[k] for k in zeros) <= 1 - g[j])
    return

In [ ]:
#| export
def mult_selection(model, x, a, b):
    """mult_selection -- add piecewise relation with multiple selection formulation
    Parameters:
        - model: a model where to include the piecewise linear relation
        - a[k]: x-coordinate of the k-th point in the piecewise linear relation
        - b[k]: y-coordinate of the k-th point in the piecewise linear relation
    Returns the model with the piecewise linear relation on added variables x, f, and z.
    """
    K = len(a) - 1
    y, z = {}, {}
    for k in range(K):
        y[k] = model.addVar(
            lb=-GRB.INFINITY
        )  # do not name variables for avoiding clash
        z[k] = model.addVar(vtype="B")
    # x = model.addVar(lb=a[0], ub=a[K], vtype="C")
    f = model.addVar(lb=-GRB.INFINITY)
    model.update()

    for k in range(K):
        if k > 0:
            model.addConstr(y[k] >= a[k] * z[k])
        if k + 1 < K:
            model.addConstr(y[k] <= a[k + 1] * z[k])

    model.addConstr(quicksum(z[k] for k in range(K)) == 1)
    model.addConstr(x == quicksum(y[k] for k in range(K)))

    c = [float(b[k + 1] - b[k]) / (a[k + 1] - a[k]) for k in range(K)]
    d = [b[k] - c[k] * a[k] for k in range(K)]
    model.addConstr(f == quicksum(d[k] * z[k] + c[k] * y[k] for k in range(K)))

    return f, z

## multidict関数

multidict関数は，辞書を引数として入力すると，
第1の返値としてキーのリスト，2番目以降の返値として辞書を返す．

In [ ]:
#| export
def multidict(dic):
    keylist = list(dic.keys())
    temp = dic[keylist[0]]
    if isinstance(temp, list):
        retList = []
        for i in range(len(temp)):
            retList.append({})
        for k in keylist:
            for i in range(len(temp)):
                retList[i][k] = dic[k][i]
        retList.insert(0, keylist)
        return tuple(retList)
    else:
        retDict = {}
        for k in keylist:
            retDict[k] = dic[k]
        return keylist, retDict

In [ ]:
I,d = multidict({1:80, 2:270, 3:250 , 4:160, 5:180})
J,M,N = multidict({1:[500,600], 2:[800,500], 3:[500,100]})
print(I,d)
print(J,M,N)

[1, 2, 3, 4, 5] {1: 80, 2: 270, 3: 250, 4: 160, 5: 180}
[1, 2, 3] {1: 500, 2: 800, 3: 500} {1: 600, 2: 500, 3: 100}


## tuplelistクラス

tuplelistは，名前の通りタプル（組）を要素としたリストである．

tuplelistには，引数によって指定した条件を満たす要素を取り出すselectメソッドが
準備されている．

In [ ]:
#| export
class tuplelist(list):
    """
    Tuplelist class for mypulp.py
    It works like Gurobi's tuplelist
    """

    def select(self, *args):
        """
        select elements from the tuplelist that matches the arguments
        """
        ret = []
        for tpl in self:
            for i, element in enumerate(tpl):
                if args[i] == element or args[i] == "*":
                    pass
                else:
                    break
            else:
                ret.append(tpl)
        return ret

In [ ]:
cost = {(1,1):4,  (1,2):6, (1,3):9,
        (2,1):5,  (2,2):4, (2,3):7,
        (3,1):6,  (3,2):3, (3,3):4,
        (4,1):8,  (4,2):5, (4,3):3,
        (5,1):10, (5,2):8, (5,3):4
        } 
arcs = tuplelist([(i,j) for (i,j) in cost])
print( arcs.select(2,"*") )

[(2, 1), (2, 2), (2, 3)]


## 線形表現クラス

In [ ]:
#| export
class LinExpr(LpAffineExpression):
    """
    Linear Expresstion Class 
    """
    def __init__(self, coeff=[], var=[]):
        if len(coeff) != len(var):
            print("Lengths of coefficient and variable lists must be equal!")
            raise TypeError
        # temp=[]
        temp = zip(var, coeff)
        LpAffineExpression.__init__(self, temp)

    def addTerms(self, coeff, var):
        self.addterm(var, coeff)
        
    def size(self):
        return len(self)
        

## 変数クラス

In [ ]:
#| export
class Variable(LpVariable):
    """
    Variable Class
    """
    def __init__(self, name="", lowBound=0.0, upBound=GRB.INFINITY, cat=LpContinuous):
        LpVariable.__init__(
            self, name=name, lowBound=lowBound, upBound=upBound, cat=cat
        )
        self.ub = upBound
        self.lb = lowBound
        self.name = name

    def __repr__(self):
        return (
            "Variable: "
            + self.name
            + " lb="
            + str(self.lb)
            + " ub="
            + str(self.ub)
            + " vtype="
            + str(self.vtype)
            + " X="
            + str(self.X)
            + " RC="
            + str(self.RC)
        )

## CBC options

Coin Branch and Cutを用いて最適化する際に，以下のオプションを付加することができる．

```
class pulp.apis.PULP_CBC_CMD(mip=True, msg=True, timeLimit=None, fracGap=None, maxSeconds=None, gapRel=None, gapAbs=None, presolve=None, cuts=None, strong=None, options=None, warmStart=False, keepFiles=False, path=None, threads=None, logPath=None, mip_start=False)
Bases: pulp.apis.coin_api.COIN_CMD
```

This solver uses a precompiled version of cbc provided with the package

Parameters

- mip (bool) – if False, assume LP even if integer variables

- msg (bool) – if False, no log is shown

- timeLimit (float) – maximum time for solver (in seconds)

- gapRel (float) – relative gap tolerance for the solver to stop (in fraction)

- gapAbs (float) – absolute gap tolerance for the solver to stop

- threads (int) – sets the maximum number of threads

- options (list) – list of additional options to pass to solver

- warmStart (bool) – if True, the solver will use the current value of variables as a start

- keepFiles (bool) – if True, files are saved in the current directory and not deleted after solving

- path (str) – path to the solver binary

- logPath (str) – path to the log file

- presolve (bool) – if True, adds presolve on

- cuts (bool) – if True, adds gomory on knapsack on probing on

- strong (bool) – if True, adds strong

モデルクラスの最適化メソッド optimize の引数 solverで， PULP_CBC_CMDのインスタンスを渡す．
```python
 optimize(self, solver=PULP_CBC_CMD(timeLimit=max_cpu, presolve=True)) 
```

## モデルクラス

In [ ]:
#| export
class Model(LpProblem):
    """
    Model Class
    """
    def __init__(self, name=""):
        super().__init__(name)
        self.var_id = 0
        self.constr_id = 0

    class Params:
        OutputFlag = 0

    def addSOS(self, sos_type, var_list=[]):
        if sos_type == 1:
            dummy_list = []
            for i in range(len(var_list)):
                dummy_list.append(self.addVar(vtype="B"))
            self.addConstr(quicksum(d for d in dummy_list) <= 1)
            for i in range(len(var_list)):
                self.addConstr(
                    var_list[i] <= min(var_list[i].upBound, GRB.MAXINT) * dummy_list[i]
                )
        elif sos_type == 2:
            # assume that sum x_i <=1
            convex_comb_agg_log(self, var_list)
        #            dummy_list=[]
        #            for i in range(len(var_list)-1):
        #                dummy_list.append( self.addVar(vtype="B") )
        #            self.addConstr( quicksum( d for d in dummy_list) <=1 )
        #            for i in range(len(var_list)-1):
        #                self.addConstr( var_list[i]+var_list[i+1] <= dummy_list[i] )
        else:
            print("SOS Type Error")
            raise TypeError

    def setPWLObj(self, var, x, y):
        f, z = mult_selection(self, var, x, y)
        self.piece += f

    def addVar(self, lb=0.0, ub=GRB.INFINITY, vtype="C", name=""):
        if name == "":
            self.var_id += 1
            name = "x_{0}".format(self.var_id)

        for i, s in [(lb, "lb"), (ub, "ub")]:
            if not isinstance(i, float) and not isinstance(i, int):
                print(f"{s} must be float or integer")
                raise TypeError

        if vtype == "C":
            CAT = LpContinuous
            var = Variable(name=name, lowBound=lb, upBound=ub, cat=CAT)
        elif vtype == "I":
            CAT = LpInteger
            var = LpVariable(name=name, lowBound=lb, upBound=ub, cat=CAT)
        elif vtype == "B":
            CAT = LpInteger
            var = LpVariable(name=name, lowBound=0, upBound=1, cat=CAT)
        elif vtype == "S":  # semi continuous
            CAT = LpContinuous
            if lb != 0:
                var = LpVariable(name=name, lowBound=0, upBound=ub, cat=CAT)
                dummy01 = self.addVar(vtype="B")
                self.addConstr(var >= dummy01 * lb)
                self.addConstr(var <= min(ub, GRB.MAXINT) * dummy01)
            else:
                var = LpVariable(name=name, lowBound=lb, upBound=ub, cat=CAT)
        elif vtype == "N":  # semi integer
            CAT = LpInteger
            if lb != 0:
                var = LpVariable(name=name, lowBound=0, upBound=ub, cat=CAT)
                dummy01 = self.addVar(vtype="B")
                self.addConstr(var >= dummy01 * lb)
                self.addConstr(var <= min(ub, GRB.MAXINT) * dummy01)
            else:
                var = LpVariable(name=name, lowBound=lb, upBound=ub, cat=CAT)
        else:
            print("vtype must be 'C', 'I', 'B', 'S', or 'N'")
            raise TypeError
        var.vtype = vtype
        var.X = None
        var.RC = None
        return var

    def update(self):
        pass

    def relax(self):
        for v in self.variables():
            v.cat = LpContinuous
        return self

    def addConstr(self, Constraint=None, name="", lhs=None, sense=None, rhs=0):
        if name == "":
            self.constr_id += 1
            name = "c_{0}".format(self.constr_id)

        if sense is None:
            self += Constraint, name  # add a constraint
        else:
            if sense == "<" or sense == "<=":
                lpSense = LpConstraintLE
            elif sense == ">" or sense == ">=":
                lpSense = LpConstraintGE
            else:
                sense = LpConstraintEQ
            Constraint = LpConstraint(e=lhs, sense=lpSense, rhs=rhs)
            self += Constraint, name
        Constraint.ConstrName = name
        Constraint.Pi = None
        Constraint.Slack = None
        return Constraint

    def setObjective(self, obj, sense, name=""):
        self += obj, name
        self.sense = sense

    def optimize(self, solver=None, **kwargs):

        try:
            _ = self.objective.name
        except AttributeError:
            self.setObjective(0.0, GRB.MINIMIZE)

        status = self.solve(solver, **kwargs)
        if status == 1:
            self.Status = GRB.Status.OPTIMAL

            self.ObjVal = value(self.objective)
            for v in self.variables():
                v.VarName = v.name
                v.X = v.varValue
                v.RC = v.dj

            for c in self.constraints:
                self.constraints[c].Pi = self.constraints[c].pi
                self.constraints[c].Slack = self.constraints[c].slack
                self.constraints[c].ConstrName = self.constraints[c].name

        elif status == -1:
            self.Status = GRB.Status.INFEASIBLE
        elif status == -2:
            self.Status = GRB.Status.UNBOUNDED
        else:
            # Not Solved = 0 or Undefined= -3
            self.Status = GRB.Status.UNDEFINED

    def getVars(self):
        return self.variables()

    def getConstrs(self):
        ret = [self.constraints[c] for c in self.constraints]
        return ret

    def write(self, filename):
        if filename[-2:] == "lp":
            self.writeLP(filename)
        elif filename[-3:] == "mps":
            self.writeMPS(filename)

## 例題

In [ ]:
TEST=False
if TEST:
    model = Model("lo1")

    J, v = multidict({1: 16, 2: 19, 3: 23, 4: 28})

    # x1 = model.addVar(vtype="C", name="x1")
    # x2 = model.addVar(vtype="I", name="x2")
    # x3 = model.addVar(lb=0, ub=30, vtype="B", name="x3")

    x1 = model.addVar(vtype=GRB.CONTINUOUS, name="x1")
    # x1 = model.addVar(vtype="N", lb=10.5, ub=50,name="x1")
    x2 = model.addVar(vtype="C", name="x2")
    x3 = model.addVar(lb=0, ub=30, vtype="C", name="x3")

    model.update()
    #model.addSOS(2, [x1, x2, x3])
    #L1 = LinExpr([2, 1, 1], [x1, x2, x3])
    # L1=LinExpr()
    # L1.addTerms(2,x1)
    # L1.addTerms(1,x2)
    # L1.addTerms(1,x3)

    #model.addConstr(lhs=L1,sense="<=",rhs=60,name="C0")
    model.addConstr(x1 + 2*x2 + x3 <= 60, name="C1")

    #c1 = model.addConstr(lhs=L1, sense="<=", rhs=60)
    model.addConstr(x1 + 2 * x2 + x3 <= 60)

    model.setObjective(15 * x1 + 18 * x2 + 30 * x3, GRB.MAXIMIZE)

    #model.write("mypulp1.mps")
    #model.write("mypulp1.lp")
    
    #SCIPを使用する場合
    #solver = pulp.SCIP(timeLimit=1)
    #model.optimize(solver=solver)
    
    #model.optimize()

    if model.Status == GRB.Status.OPTIMAL:
        model.write("answer.sol")
        print("Opt. Value=", model.ObjVal)
        # for v in model.getVars():
        #    print(v.VarName,v.X,v.RC)
        for v in model.getVars():
            print(v.VarName, v.X)
        for c in model.getConstrs():
            print(c.ConstrName, c.Pi)
    # m=Model()
    # x=m.addVar( name="x", vtype="I" , lb=1)
    # y=m.addVar( name="y", vtype="I" , lb=1)
    # z=m.addVar( name="z", vtype="I" , lb=1)
    # m.update()
    # m.addConstr(2*x+ y ==11)
    # m.addConstr(y+ 3*z ==14)
    # # m.setObjective(0,GRB.MINIMIZE)
    # m.optimize()
    # print(x.X,y.X,z.X)

Opt. Value= 1350.0
x1 30.0
x2 0.0
x3 30.0
C1 None
c_1 None


In [ ]:
if TEST:
    m=Model()

    x = [0,2,3,1]
    y = [1,0,3,3]

    n=len(x)

    X=m.addVar(ub=3.0, name="X")
    Y=m.addVar(ub=3.0, name ="Y")

    Z=m.addVar(name ="Z")

    a,b={},{}
    for i in range(n):
        a[i] =m.addVar(name="a({0})".format(i))
        b[i] =m.addVar(name="b({0})".format(i))
        m.update()

    for i in range(n):
        m.addConstr( a[i] >= x[i] -X )
        m.addConstr( a[i] >= X-x[i] )
        m.addConstr( b[i] >= y[i] -Y )
        m.addConstr( b[i] >= Y-y[i] )

        m.addConstr(Z>=a[i]+b[i])

    m.setObjective(Z, GRB.MINIMIZE)
    # m.setObjective( quicksum( 1*a[i] for i in range(n))
    #                + quicksum( 1*b[i] for i in range(n)), GRB.MINIMIZE )


    m.optimize()

    print(X.X,Y.X)
    print(m.ObjVal)
    for i in range(n):
        print(a[i].X,b[i].X)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mikiokubo/Library/Caches/pypoetry/virtualenvs/scmopt-KVX_UVCf-py3.8/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/69/5y96sdc94jxf6khgc8mlmxrr0000gn/T/a9600581b8304017ab19ab021b6e0065-pulp.mps branch printingOptions all solution /var/folders/69/5y96sdc94jxf6khgc8mlmxrr0000gn/T/a9600581b8304017ab19ab021b6e0065-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 25 COLUMNS
At line 71 RHS
At line 92 BOUNDS
At line 104 ENDATA
Problem MODEL has 20 rows, 11 columns and 44 elements
Coin0008I MODEL read with 0 errors
Presolve 10 (-10) rows, 6 (-5) columns and 26 (-18) elements
0  Obj 0 Primal inf 12.999995 (5)
6  Obj 2.5
Optimal - objective value 2.5
After Postsolve, objective 2.5, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 2.5 - 6 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to a

In [ ]:
if TEST:
    model = Model("lo1")

    x = model.addVar(name='tonkoke')
    y = model.addVar(name='kokenton')
    z = model.addVar(name='mix')

    model.setObjective(15*x + 18*y + 30*z, GRB.MAXIMIZE)
    model.addConstr(2*x +   y + z <= 60, name='pork')
    model.addConstr(  x + 2*y + z <= 60, name='chicken')
    model.addConstr(            z <= 30, name='beef')

    model.optimize()
    print( model.ObjVal )
    for i in model.getVars():
        print(i.VarName, i.X)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mikiokubo/Library/Caches/pypoetry/virtualenvs/scmopt-KVX_UVCf-py3.8/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/69/5y96sdc94jxf6khgc8mlmxrr0000gn/T/3175126010324092b3a9adfcf59ef2f7-pulp.mps max branch printingOptions all solution /var/folders/69/5y96sdc94jxf6khgc8mlmxrr0000gn/T/3175126010324092b3a9adfcf59ef2f7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 19 RHS
At line 23 BOUNDS
At line 27 ENDATA
Problem MODEL has 3 rows, 3 columns and 7 elements
Coin0008I MODEL read with 0 errors
Presolve 2 (-1) rows, 3 (0) columns and 6 (-1) elements
0  Obj -0 Dual inf 62.999997 (3)
2  Obj 1230
Optimal - objective value 1230
After Postsolve, objective 1230, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1230 - 2 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
